In [1]:
!pip install pycld2

    100% |████████████████████████████████| 41.4MB 1.3MB/s  eta 0:00:01
  Running setup.py bdist_wheel for pycld2 ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import time

import pandas as pd
import pycld2 as cld2
from pandarallel import pandarallel
from sklearn.model_selection import train_test_split

pandarallel.initialize()

INFO: Pandarallel will run on 72 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
start = time.time()
df = pd.read_json("s3://yelp-dataset-pt-9/data/json/review.json", orient="records", lines=True)
time.time() - start

271.8896059989929

In [4]:
print(df.shape)
df.head()

(8021122, 9)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


In [5]:
df.isna().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [6]:
df = df.dropna()
df.shape

(8021122, 9)

In [7]:
def check_language(review):
    try:
        isReliable, textBytesFound, details = cld2.detect(review)
        if (isReliable) & (details[0][1] == 'en'):
            return True
        return False
    except:
        return False

In [8]:
start = time.time()
en = df.loc[df["text"].parallel_apply(check_language), ["text", "stars"]]
time.time() - start

# 244.31 seconds without parallel

33.69557452201843

In [9]:
en.shape

(7983028, 2)

In [10]:
len(df) - len(en)

38094

In [11]:
en.head()

,text,stars
0,"As someone who has worked with many museums, I...",2
1,I am actually horrified this place is still in...,1
2,I love Deagan's. I do. I really do. The atmosp...,5
3,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",1
4,"Oh happy day, finally have a Canes near my cas...",4


In [13]:
train, test = train_test_split(en, test_size=.2, stratify=en['stars'], random_state=42)
test, val = train_test_split(test, test_size=.5, stratify=test['stars'], random_state=42)

In [14]:
len(train), len(test), len(val)

(6386422, 798303, 798303)

In [15]:
train.to_csv("s3://yelp-dataset-pt-9/spencer/data/csv/en/train.csv", index=False, header=['text', 'labels'])
test.to_csv("s3://yelp-dataset-pt-9/spencer/data/csv/en/test.csv", index=False, header=['text', 'labels'])
val.to_csv("s3://yelp-dataset-pt-9/spencer/data/csv/en/val.csv", index=False, header=['text', 'labels'])